# Hyperparameter Tuning using HyperDrive

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model
from azureml.core import Environment, ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import ScriptRunConfig
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import shutil
import requests
import json


## Setup

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone'
experiment=Experiment(ws, experiment_name)


In [3]:
#because of Udacity virtual machine time constraints, I'm creating a new compute cluster
amlcompute_cluster_name = "capstonecompute2"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


Creating
Succeeded...............................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

The dataset used for the capstone project is the Heart Failure Clinical Records dataset that can be found at the UCI Machine Learning Repository. The dataset will be used for classification to help predict a death event.

In [4]:
# test to see if dataset is in store
key = 'heartfailuredataset'
if key in ws.datasets.keys(): 
    dataset = ws.datasets[key] 
    print("Found dataset")

# if not, load the dataset, save it to the store
else:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(workspace=ws, name=key)
    

Found dataset


## Setting up the environment and training script

### Creating Environment File

In [5]:
my_env = CondaDependencies()
my_env.add_conda_package("scikit-learn")
with open("conda_dependencies.yml", "w") as f:
    f.write(my_env.serialize_to_string())

### Creating training script

As the task is binary classification, I chose to use Scikit-Learn logistic regression as I am most familiar with logistic regression when it comes to supervised learning binary classifiers and have experience with using SciKit-Learn from project 1 of the nanodegree.

In [6]:
%%writefile train.py

from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import argparse
import os
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory

run = Run.get_context()

def main():
    
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    data = TabularDatasetFactory.from_delimited_files(url)
    x = data.to_pandas_dataframe()
    y = x.pop("DEATH_EVENT")    
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=625, shuffle=True)

    parser = argparse.ArgumentParser()

    parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength. Smaller values cause stronger regularization")
    parser.add_argument('--max_iter', type=int, default=100, help="Maximum number of iterations to converge")
    parser.add_argument('--solver', type=str, default='lbfgs', help="chose the algorithm to train the model")

    args = parser.parse_args()

    run.log("Regularization Strength:", np.float(args.C))
    run.log("Max iterations:", np.int(args.max_iter))
    run.log("Algorithm: ", args.solver)

    # creates the logistic regression model
    model = LogisticRegression(solver=args.solver, C=args.C, max_iter=args.max_iter).fit(x_train, y_train)

    # gets and logs the accuracy
    accuracy = model.score(x_test, y_test)
    run.log("Accuracy", np.float(accuracy))
    
    # dumps the run
    os.makedirs('outputs', exist_ok=True)
    joblib.dump(model,'outputs/model.joblib')

if __name__ == '__main__':
    main()    

Writing train.py


## Hyperdrive Configuration

The three hyperparameters I chose to use for tuning are: C, the inverse of the regularization parameter, max_iter, the maximum number of iterations to allow for the solvers to converge, and solver, the solver used.  Other possible hyperparameters relied on what solver was chosen and hence not included. See readme.md for reasons for choices of C and max_iter.
RandomParameterSampling was chosen instead of Grid as it uses less time and resources and produces almost as good hyperparameters as using Grid.
A MedianStoppingPolicy was chosen with an evaluation_interval of 1 and delay_evaluation of 5 as this is a conservative policy without loss on the primary metric, according to Microsoft documentation.
The training is done on the local compute target.
Accuracy was chosen as the primary metric (although thinking back I should have used recall) with a goal to maximize accuracy.
To keep training short, max_total_runs was set to 50 and max_duration_minutes was set to 30.


In [7]:
# Specify parameter sampler
sample_space = {
    'C': choice(0.01, 0.1, 1, 10, 100),
    'max_iter' : choice(50,75,100,125,150,175,200),
    'solver' : choice('liblinear','sag','lbfgs', 'saga')
}
ps = RandomParameterSampling(sample_space)

# Specify a Policy
policy = MedianStoppingPolicy(evaluation_interval=1,delay_evaluation=5)

# creating a training directory and copying train.py to that directory
if "training" not in os.listdir():
    os.mkdir("./training")
shutil.copy('./train.py','./training')

#load environment #need to see if need to get rid of /envs/ in filepath
sklearn_env = Environment.from_conda_specification(name="sklearn-env",
                                                  file_path="./conda_dependencies.yml")

# Create a SKLearn estimator for use with train.py
est = ScriptRunConfig(source_directory='./training',
                     script='train.py',
                     compute_target=compute_target,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    max_total_runs=50,
                                    max_duration_minutes=30,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [8]:
#TODO: Submit your experiment
run = experiment.submit(hyperdrive_config)

## Run Details

In [9]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
status = run.wait_for_completion()

## Best Model

In [11]:
best_run = run.get_best_run_by_primary_metric()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone,HD_43d1b4c2-5c43-47b3-a44e-0cfb6bb2cbba_8,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
best_run.get_details()['runDefinition']['arguments']

['--C', '1', '--max_iter', '50', '--solver', 'lbfgs']

In [13]:
best_run.get_metrics(name='Accuracy')

{'Accuracy': 0.8133333333333334}

### Register the model

In [14]:
# set the model properties
properties = {"model":"scikit-lean logistic regression",
             "hyperparameters":str(best_run.get_details()['runDefinition']['arguments']),
             "accuracy":str(best_run.get_metrics(name='Accuracy'))}

In [15]:
# register the model
model = best_run.register_model('best_hd_run', model_path = 'outputs/model.joblib',properties=properties)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

### Create a scoring script


In [15]:
%%writefile score.py

import os
import numpy as np
import json
import joblib
from azureml.core.model import Model

def init():
    global model
    try:
        model_path = Model.get_model_path('best_hd_run')
        model = joblib.load(model_path)
    except Exception as err:
        print("init method error: "+str(err))

def run(data):
    try:
        #data = np.array(json.loads(data))
        data = json.loads(data)
        strn = "extracted json\n"
        data = np.array(data["data"])
        strn += "converted data to np array\n"
        result = model.predict(data)
        strn += "sent the data to the model for prediction\n"
        print(strn)
        return result.tolist()
    except Exception as err:
        return strn+"run method error: "+str(err)

Writing score.py


### Create inference and deployment configurations

In [16]:
# create inference_config
inference_config = InferenceConfig(entry_script="score.py", environment = sklearn_env)

In [17]:
#set deployment_config
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

### Deploy the model

In [ ]:
#deploy the model
service=Model.deploy(workspace=ws,
                    name="mwebservice",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

In [ ]:
service.get_logs()

## Call the endpoint

In [20]:
# get the data to send
ds = dataset.to_pandas_dataframe()
train, test = train_test_split(ds, random_state=625, shuffle=True)
tosend = test[:2].values.tolist()
tosend = [tosend[0][:-1],tosend[1][:-1]]

In [ ]:
test.head()

In [ ]:
tosend

In [24]:
url = service.scoring_uri

In [ ]:
url

In [ ]:
data = json.dumps({"data":tosend})
headers = {'Content-Type':'application/json'}

response = requests.post(url,data,headers=headers)
print(response.text)

In [ ]:
service.get_logs()

### Remove the service and shut down the computer cluster

In [27]:
service.delete()

In [ ]:
compute_target.delete()